In [ ]:
# 爬取數據新聞新聞記者名單和信息
import csv
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time


def get_dynamic_page_script(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(2)
    h = browser.find_element_by_css_selector('html')
    t = h.get_attribute('innerHTML')
    browser.quit()
    return t
#這個網站用requests拿不到腳本，因此用selenium


def scrape_directory(page_script):
    mypage=BeautifulSoup(page_script,"html.parser")
    journalists_data = []
    for i in mypage.find_all('div', attrs={'class':"back"}):
        journalist = []
        journalist.append(i.find('div', attrs={'class':"name"}).text)
        journalist.append(i.find('div', attrs={'class':"institution"}).text)
        journalist.append(i.find('div', attrs={'class':"city"}).text)
        journalist.append(i.find('div', attrs={'class':"github"}).a['href'])
#         journalist.append(i.find('div', attrs={'class':"pgp"}).a.text)
#         journalist.append(i.find('div', attrs={'class':"email"}).a['href'].replace('mailto:', ''))
#         journalist.append(i.find('div', attrs={'class':"twitter"}).a['href'])
#         journalist.append(i.find('div', attrs={'class':"website"}).a['href'])
# 經測試，動態爬尋的速度太慢了，最後還是選擇用靜態爬，此處供以後參考
#     for i in browser.find_elements_by_class_name("back"):
#         journalist = []
#         journalist.append(i.find_element_by_class_name("name").text)
#         journalist.append(i.find_element_by_class_name("institution").text)
#         journalist.append(i.find_element_by_class_name("city").text)
#         journalist.append(i.find_element_by_xpath("//div[@class='pgp']/a").text)
#         journalist.append(i.find_element_by_xpath("//div[@class='email']/a").get_attribute('href').replace('mailto:', ''))
#         journalist.append(i.find_element_by_xpath("//div[@class='twitter']/a").get_attribute('href'))
#         journalist.append(i.find_element_by_xpath("//div[@class='github']/a").get_attribute('href'))
#         journalist.append(i.find_element_by_xpath("//div[@class='website']/a").get_attribute('href'))
#     journalists_data.append(journalist)
    
    #因為原網站已經有N/A了，所以在這裡把空信息轉化成N/A，統一格式
        for i, value in enumerate(journalist):
            if value == '':
                journalist[i]='N/A'
        journalists_data.append(journalist)
    return journalists_data



url = 'http://jplusplus.github.io/global-directory/'
page_script = get_dynamic_page_script(url)
journalists_data = scrape_directory(page_script)
with open('journalists-data.csv','w') as f:
    csv.writer(f).writerows(journalists_data)
# 輸出完整的記者基本信息表csv

In [1]:
%%time
# 根據之前導出的csv中每位記者的github網址，爬取他們的contribution歷史
import csv
import requests
from bs4 import BeautifulSoup
import datetime 
import time


def scrape_github_contributions(homepage_url):
#今後只要有一個人的github的homepage_url,就可以爬取所有contribution了
    def scrpe_year_urls(homepage_url):
        year_urls = [homepage_url] 
        #2018年（今年）在homepage上可以截止到今天，但如果載入html中的地址，會截止到今年12-31（未來），所以先把首頁加進去
        r = requests.get(homepage_url).text
        home_page_script = BeautifulSoup(r,"html.parser")
        for i in home_page_script.find_all('a',attrs={'class':"js-year-link filter-item px-3 mb-2 py-2 "}):
        #這個class不包含2018年（今年）
            url ='{}{}'.format('https://github.com',i['href'])
            year_urls.append(url)
        return year_urls
    
    def scrape_one_year_contributions(url):
        r = requests.get(url).text
        page_script = BeautifulSoup(r,"html.parser")
        all_rects = page_script.find_all('rect')
        for i in reversed(all_rects):
            contributions[i['data-date']] = i['data-count']
    # 因為不同年份的每一頁在12.30這一天顯示有重複（每一年都會顯示到去年的12.30左右），所以用dict的key的不可重複性避免這個問題   
            
    contributions = {} 
    for i in scrpe_year_urls(homepage_url): 
        scrape_one_year_contributions(i)
    contributions_list = list(contributions.values())
    #dict.values輸出的不是list，要重新列表化，方便之後輸出
    return contributions_list


def write_head(beginDate): #添加標題行
    def dateRange(beginDate, endDate): #引用網絡上的，黑箱！
        dates = []
        dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
        date = beginDate[:]
        while date <= endDate:
            dates.append(date)
            dt = dt + datetime.timedelta(1)
            date = dt.strftime("%Y-%m-%d")
        return reversed(dates)
    #return一個從現在到以前的日期列表，用來作為csv文件的head的一部分，之後每一個日期的數據只佔一格
#     head = ["name","institution","city","pgp","email","twitter","website","github"]
    head = ["name","institution","city","github"]
    today = time.strftime("%Y-%m-%d")
    head.extend(dateRange(beginDate, today))
    return head



with open('journalists-data.csv','r') as f:  #讀取上一部分輸出的記者基本信息表
    with open('journalists-data-including-github-contributions.csv','a') as d:
        csv.writer(d).writerow(write_head('2007-12-30')) #github是08年發布的，但顯示最早到20171230，所以從這個日期開始爬
        for row in csv.reader(f):
            try:
                row.extend(scrape_github_contributions(row[3])) #row[3]是每位記者的github地址
                csv.writer(d).writerow(row)
            except:
                row.append('N/A') 
                #疑問：對於沒有github的記者，應該所有日期的contribution全部標註為N/A還是0？這個可以之後再定
                csv.writer(d).writerow(row)

CPU times: user 56.2 s, sys: 1.34 s, total: 57.5 s
Wall time: 25min 55s


In [95]:
import pandas as pd
import numpy as np
#显示所有列
pd.set_option('display.max_columns', 25)
#显示所有行
pd.set_option('display.max_rows', 20)
df=pd.read_csv('journalists-data-including-github-contributions.csv')
df=df[np.isnan(df['2018-11-04']) == False] #2018-11-04如果為N/A，則說明此人沒有github
df.to_csv('Result.csv') #寫出的文件中contributions是float，還不知道怎麼變成整數
df

,name,institution,city,github,2018-11-04,2018-11-03,2018-11-02,2018-11-01,2018-10-31,2018-10-30,2018-10-29,2018-10-28,...,2008-01-10,2008-01-09,2008-01-08,2008-01-07,2008-01-06,2008-01-05,2008-01-04,2008-01-03,2008-01-02,2008-01-01,2007-12-31,2007-12-30
1,Justin Myers,The Associated Press,"United States, Brooklyn (US-VA)",http://github.com/myersjustinc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Annabel Church,NaN,"N/A, Germany",http://www.github.com/arc64,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mina Demian,Africa Check,"Johannesburg, South Africa",http://www.github.com/minademian,8.0,2.0,14.0,8.0,3.0,0.0,8.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Martín González,Freelance,"Barcelona, Spain",https://www.github.com/martgnz,0.0,2.0,19.0,45.0,18.0,4.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Adolfo Antón Bravo,Infotics,"Madrid, Spain",http://www.github.com/flowsta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Michael Mott,The Willits News,"Willits, United States",http://www.github.com/mwmott,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Troy Thibodeaux,The Associated Press,"New Orleans, United States",http://www.github.com/tthibo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,John Heasly,The Register-Guard,"Eugene, United States",http://www.github.com/jheasly,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Tom Cardoso,The Globe and Mail,"Toronto, Canada",http://www.github.com/tomcardoso,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Jeremia Kimelman,Code for America,"San Francisco, United States",http://www.github.com/jeremiak,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
column_list=df.columns.values.tolist()[8:]
column_list


df.loc[:,list_one_month].T.sum()

['2018-10-31',
 '2018-10-30',
 '2018-10-29',
 '2018-10-28',
 '2018-10-27',
 '2018-10-26',
 '2018-10-25',
 '2018-10-24',
 '2018-10-23',
 '2018-10-22',
 '2018-10-21',
 '2018-10-20',
 '2018-10-19',
 '2018-10-18',
 '2018-10-17',
 '2018-10-16',
 '2018-10-15',
 '2018-10-14',
 '2018-10-13',
 '2018-10-12',
 '2018-10-11',
 '2018-10-10',
 '2018-10-09',
 '2018-10-08',
 '2018-10-07',
 '2018-10-06',
 '2018-10-05',
 '2018-10-04',
 '2018-10-03',
 '2018-10-02',
 '2018-10-01',
 '2018-09-30',
 '2018-09-29',
 '2018-09-28',
 '2018-09-27',
 '2018-09-26',
 '2018-09-25',
 '2018-09-24',
 '2018-09-23',
 '2018-09-22',
 '2018-09-21',
 '2018-09-20',
 '2018-09-19',
 '2018-09-18',
 '2018-09-17',
 '2018-09-16',
 '2018-09-15',
 '2018-09-14',
 '2018-09-13',
 '2018-09-12',
 '2018-09-11',
 '2018-09-10',
 '2018-09-09',
 '2018-09-08',
 '2018-09-07',
 '2018-09-06',
 '2018-09-05',
 '2018-09-04',
 '2018-09-03',
 '2018-09-02',
 '2018-09-01',
 '2018-08-31',
 '2018-08-30',
 '2018-08-29',
 '2018-08-28',
 '2018-08-27',
 '2018-08-

In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
df_overall = pd.DataFrame(df.T.sum())

df_T_sum=df.T.sum()
df_sum

1       1147.0
4        149.0
7       1982.0
8       7951.0
11       792.0
14       582.0
15        81.0
16      1910.0
17       857.0
18      6396.0
        ...   
190        2.0
192      444.0
196      183.0
202        5.0
205      123.0
206     3795.0
207      912.0
208     6601.0
209    10282.0
213      503.0
Length: 81, dtype: float64